# Conv2D

In [11]:
import numpy as np
from tensorflow import keras

# define input data
# data random array (3, 4)
data = [[1, 2, 3, 4], 
		[5, 6, 7, 8], 
		[9, 10, 11, 12]]

data = np.array(data)
data = data.reshape(1, 3, 4, 1)
data

array([[[[ 1],
         [ 2],
         [ 3],
         [ 4]],

        [[ 5],
         [ 6],
         [ 7],
         [ 8]],

        [[ 9],
         [10],
         [11],
         [12]]]])

In [6]:
# Shapes
detector = [[[[0.5]]],
            [[[0.5]]]]
np.array(detector).shape


detector = [[0.5, 1],
            [0.5, 1]]
np.array(detector).shape


/Users/valentin/opt/anaconda3/envs/mycondaenv/lib/python3.6/site-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


(2,)

In [7]:
# SAMPLE ONE FILTER

# create sample_model
sample_model = keras.sample_models.Sequential()
sample_model.add(keras.layers.Conv2D(filters=1, kernel_size=(2, 1), input_shape=(3, 4, 1)))

# define a vertical line detector
detector = [[[[0.5]]],
            [[[0.5]]]] # 1 detector (2, 1, 1, 1) (time, features, channels, unknown)
weights = [np.array(detector), np.array([0.0])]
# for every any application of the filter we will add the bias!

# store the weights in the sample_model
sample_model.set_weights(weights)
# confirm they were stored
print(sample_model.get_weights())

[array([[[[0.5]]],


       [[[0.5]]]], dtype=float32), array([0.], dtype=float32)]


# Example Pipeline

In [21]:
import numpy as np
from tensorflow import keras

# define input data
# data random array (3, 4)
data = [[1, 2, 3, 4], 
		[5, 6, 7, 8], 
		[9, 10, 11, 12]]

data = np.array(data)
data = data.reshape(1, 3, 4, 1)
data

array([[[[ 1],
         [ 2],
         [ 3],
         [ 4]],

        [[ 5],
         [ 6],
         [ 7],
         [ 8]],

        [[ 9],
         [10],
         [11],
         [12]]]])

In [22]:
# SAMPLE 3 FILTERS

# create model_01
model_01 = keras.models.Sequential()
model_01.add(keras.layers.Conv2D(filters=3, kernel_size=(2, 1), input_shape=(3, 4, 1)))

# define a vertical line detector
detectors = [[[[0.5, 0, 1]]],
            [[[0.5, 1, 1]]]] # 1 detector (2, 1, 1, 1) (time, features, channels, numbers of detectors)
detectors = np.array(detectors)
print(detectors.shape)

# first filter averages values
# second filter cuts first row
# third filter averages values bigger

weights = [detectors, np.array([0.0, 0.0, 0.0])] # also add 3 biases
# for every any application of the filter we will add the bias!

# store the weights in the model_01
model_01.set_weights(weights)
# confirm they were stored
print(model_01.get_weights())

(2, 1, 1, 3)
[array([[[[0.5, 0. , 1. ]]],


       [[[0.5, 1. , 1. ]]]], dtype=float32), array([0., 0., 0.], dtype=float32)]


In [23]:


# apply filter to input data
yhat = model_01.predict(data)
print(yhat.shape)
data_after = yhat
yhat

(1, 2, 4, 3)


array([[[[ 3.,  5.,  6.],
         [ 4.,  6.,  8.],
         [ 5.,  7., 10.],
         [ 6.,  8., 12.]],

        [[ 7.,  9., 14.],
         [ 8., 10., 16.],
         [ 9., 11., 18.],
         [10., 12., 20.]]]], dtype=float32)

In [24]:
# SAMPLE 2 FILTERS

# create model_01
model_02 = keras.models.Sequential()
model_02.add(keras.layers.Conv2D(filters=2, kernel_size=(2, 1), input_shape=(2, 4, 3))) # (2, 4, 3) (time, features, channels_ac_maps)

filter_01 = [[[0, 1, 0]],
            [[0, 1, 0]]] # (2, 1, 3) (filter_time_axis, filter_features_axis, filter_channels_activation_map_axis)

filter_02 = [[[1/3, 1/3, 1/3]],
            [[1/3, 1/3, 1/3]]]

filters = np.array([np.array(filter_01), np.array(filter_02)]) # (2, 2, 4, 3) (number_of_filters, time, features, channels_activation_maps)
detectors = filters.transpose(1, 2, 3, 0) # (2, 1, 3, 2) (time, features, channels_ac_map, number_of_filters)

weights = [detectors, np.array([0.0, 0.0])] # also add 2 biases
# for every any application of the filter we will add the bias!

# store the weights in the model_02
model_02.set_weights(weights)
# confirm they were stored
print(model_02.get_weights())



[array([[[[0.        , 0.33333334],
         [1.        , 0.33333334],
         [0.        , 0.33333334]]],


       [[[0.        , 0.33333334],
         [1.        , 0.33333334],
         [0.        , 0.33333334]]]], dtype=float32), array([0., 0.], dtype=float32)]


In [26]:


# apply filter to input data
yhat = model_02.predict(data_after)
print(yhat.shape) # (1, 1, 4, 2) (batch_size, time, features, number_of_filters_channes)
yhat

(1, 1, 4, 2)


array([[[[14.      , 14.666667],
         [16.      , 17.333334],
         [18.      , 20.      ],
         [20.      , 22.666668]]]], dtype=float32)